# Convolutional Neural Networks

This note book will be used to test convolutional neural networks on each of the 3 ontologies.

In [16]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import itertools
import pprint
import yaml

In [17]:
print("TensorFlow v" + tf.__version__)
print("Numpy v" + np.__version__)

TensorFlow v2.15.0
Numpy v1.26.4


In [18]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [19]:
config_path = './config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [20]:
USE_75_PERCENT_DATA = config['use_75_percent_datasets']
partial_dataset_prefix = '75percent_' if USE_75_PERCENT_DATA else ''
BP_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_BiologicalProcesses.pkl")
CC_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_CellularComponent.pkl")
MF_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_MolecularFunction.pkl")
BP_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_BiologicalProcesses.pkl")
CC_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_CellularComponent.pkl")
MF_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_MolecularFunction.pkl")

In [21]:
train_data_dict = {'Biological Processes': [BP_train_df, BP_label_df],
                   'Cellular Component': [CC_train_df, CC_label_df],
                   'Molecular Function': [MF_train_df, MF_label_df]
}

In [22]:
num_labels = 1500
num_folds = config['num_folds']

## Model 1 Architecture: CNN

In [23]:
BATCH_SIZE = config['batch_size']

In [24]:
def model1_training(dataset_name, data):
  train = data[0]
  label = data[1]

  INPUT_SHAPE = (train.shape[1], 1)

  best_f1 = 0
  print('=======================================================================')
  print(f'Training for {dataset_name}')
  
  model_root_path = f'{config["directories"]["models"]}/CNNMod1'

  for num_filters, size_kernel in itertools.product([16, 32], [64, 128, 256]):
    print('----------------------------------------------------------------------')
    print('The number of filters is ', num_filters)
    print('The size of the kernel is ', size_kernel)

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1

    for train_fold, test_fold in kfold.split(train, label):

      model1 = tf.keras.Sequential([
          tf.keras.layers.BatchNormalization(input_shape = INPUT_SHAPE),
          tf.keras.layers.Conv1D(filters = num_filters, kernel_size = size_kernel, activation = 'relu'),
          tf.keras.layers.MaxPooling1D(pool_size = 2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(units = 512, activation = 'relu'),
          tf.keras.layers.Dense(units = num_labels, activation = 'sigmoid')
          ])

      # Compile model
      model1.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
          loss='binary_crossentropy',
          metrics=['binary_accuracy',
                  tf.keras.metrics.AUC(),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall(),
                  ] # tf.keras.metrics.F1Score() not appropriate as it is calculated batchwise
          )

      print(f'Training for fold {fold_no} ...')

      # Fit the data to the model
      history = model1.fit(
          train.iloc[train_fold], label.iloc[train_fold],
          validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
          batch_size=BATCH_SIZE,
          epochs=5
          )

      # Generate metrics
      scores = model1.evaluate(train.iloc[test_fold], label.iloc[test_fold], verbose=0)
      precision = scores[3]
      recall = scores[4]
      F1_score = 2*precision*recall / (precision + recall)
      print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model1.metrics_names[0]} of {scores[0]}; {model1.metrics_names[1]} of {scores[1]*100}%')

      if F1_score > best_f1:
        best_f1 = F1_score
        if dataset_name == 'Biological Processes':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_BP_model')
          print(f'Current best model for Biological Processes has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

        elif dataset_name == 'Molecular Function':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_MF_model')
          print(f'Current best model for Molecular Function has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

        else:
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_CC_model')
          print(f'Current best model for Cellular Component has {num_filters} filters, with a kernel size of {size_kernel} and has an F1 score of {F1_score}')

      fold_no += 1

In [25]:
for dataset in train_data_dict:
  dataset_name = dataset
  data = train_data_dict[dataset]
  model1_training(dataset_name, data)

Training for Biological Processes
----------------------------------------------------------------------
The number of filters is  16
The size of the kernel is  64
Training for fold 1 ...
Epoch 1/5
293/293 [==============================] - 9s 8ms/step - loss: 0.1001 - binary_accuracy: 0.9705 - auc_210: 0.8541 - precision_210: 0.4468 - recall_210: 0.1500 - val_loss: 0.2801 - val_binary_accuracy: 0.9742 - val_auc_210: 0.9077 - val_precision_210: 0.6735 - val_recall_210: 0.1898
Epoch 2/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0813 - binary_accuracy: 0.9747 - auc_210: 0.9147 - precision_210: 0.6701 - recall_210: 0.2196 - val_loss: 0.1044 - val_binary_accuracy: 0.9743 - val_auc_210: 0.9224 - val_precision_210: 0.6207 - val_recall_210: 0.2645
Epoch 3/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0779 - binary_accuracy: 0.9751 - auc_210: 0.9245 - precision_210: 0.6725 - recall_210: 0.2462 - val_loss: 0.0782 - val_binary_accuracy: 0.9749 - val_au

INFO:tensorflow:Assets written to: ./models/CNNMod1/best_BP_model/assets


Current best model for Biological Processes has 16 filters, with a kernel size of 64 and has an F1 score of 0.37484124798104557
Training for fold 2 ...
Epoch 1/5
293/293 [==============================] - 4s 7ms/step - loss: 0.1000 - binary_accuracy: 0.9705 - auc_211: 0.8538 - precision_211: 0.4469 - recall_211: 0.1521 - val_loss: 0.2864 - val_binary_accuracy: 0.9737 - val_auc_211: 0.9068 - val_precision_211: 0.6460 - val_recall_211: 0.2135
Epoch 2/5
293/293 [==============================] - 2s 6ms/step - loss: 0.0814 - binary_accuracy: 0.9747 - auc_211: 0.9140 - precision_211: 0.6694 - recall_211: 0.2179 - val_loss: 0.0915 - val_binary_accuracy: 0.9742 - val_auc_211: 0.9234 - val_precision_211: 0.6435 - val_recall_211: 0.2541
Epoch 3/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0780 - binary_accuracy: 0.9751 - auc_211: 0.9242 - precision_211: 0.6725 - recall_211: 0.2441 - val_loss: 0.0800 - val_binary_accuracy: 0.9742 - val_auc_211: 0.9255 - val_precision_211: 0

INFO:tensorflow:Assets written to: ./models/CNNMod1/best_BP_model/assets


Current best model for Biological Processes has 16 filters, with a kernel size of 64 and has an F1 score of 0.3891332102075608
Training for fold 4 ...
Epoch 1/5
293/293 [==============================] - 3s 7ms/step - loss: 0.0987 - binary_accuracy: 0.9707 - auc_213: 0.8609 - precision_213: 0.4601 - recall_213: 0.1585 - val_loss: 0.2631 - val_binary_accuracy: 0.9742 - val_auc_213: 0.9073 - val_precision_213: 0.7043 - val_recall_213: 0.1547
Epoch 2/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0811 - binary_accuracy: 0.9746 - auc_213: 0.9156 - precision_213: 0.6702 - recall_213: 0.2200 - val_loss: 0.0950 - val_binary_accuracy: 0.9742 - val_auc_213: 0.9209 - val_precision_213: 0.5970 - val_recall_213: 0.2806
Epoch 3/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0780 - binary_accuracy: 0.9750 - auc_213: 0.9246 - precision_213: 0.6720 - recall_213: 0.2459 - val_loss: 0.0788 - val_binary_accuracy: 0.9749 - val_auc_213: 0.9201 - val_precision_213: 0.

INFO:tensorflow:Assets written to: ./models/CNNMod1/best_BP_model/assets


Current best model for Biological Processes has 16 filters, with a kernel size of 64 and has an F1 score of 0.42444720605221126
Training for fold 9 ...
Epoch 1/5
293/293 [==============================] - 3s 6ms/step - loss: 0.0985 - binary_accuracy: 0.9707 - auc_218: 0.8625 - precision_218: 0.4596 - recall_218: 0.1599 - val_loss: 0.3044 - val_binary_accuracy: 0.9740 - val_auc_218: 0.9090 - val_precision_218: 0.6143 - val_recall_218: 0.2400
Epoch 2/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0810 - binary_accuracy: 0.9747 - auc_218: 0.9158 - precision_218: 0.6704 - recall_218: 0.2217 - val_loss: 0.0842 - val_binary_accuracy: 0.9745 - val_auc_218: 0.9227 - val_precision_218: 0.7108 - val_recall_218: 0.1757
Epoch 3/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0776 - binary_accuracy: 0.9751 - auc_218: 0.9256 - precision_218: 0.6752 - recall_218: 0.2476 - val_loss: 0.0801 - val_binary_accuracy: 0.9746 - val_auc_218: 0.9182 - val_precision_218: 0

KeyboardInterrupt: 

## Model 2 Architecture: CNN with LSTM

In [ ]:
BATCH_SIZE = 256

In [ ]:
def model2_training(dataset_name, data):
  train = data[0]
  label = data[1]

  INPUT_SHAPE = (train.shape[1], 1)

  best_f1 = 0
  print('=======================================================================')
  print(f'Training for {dataset_name}')
  
  model_root_path = f'{config["directories"]["models"]}/CNNMod2'

  for lstm_units in [2, 32]:
    print('----------------------------------------------------------------------')
    print('The number of LSTM units is ', lstm_units)

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1

    for train_fold, test_fold in kfold.split(train, label):
      forward = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
      backward = tf.keras.layers.LSTM(lstm_units, activation='relu', return_sequences=True, go_backwards=True)

      model2 = tf.keras.Sequential([
          tf.keras.layers.BatchNormalization(input_shape = INPUT_SHAPE),
          tf.keras.layers.Conv1D(filters = 32, kernel_size = 128, activation = 'relu'),
          tf.keras.layers.MaxPooling1D(pool_size = 2),
          tf.keras.layers.Bidirectional(forward, backward),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(units = 512, activation = 'relu'),
          tf.keras.layers.Dense(units = num_labels, activation = 'sigmoid')
          ])

      # Compile model
      model2.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
          loss='binary_crossentropy',
          metrics=['binary_accuracy',
                  tf.keras.metrics.AUC(),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall(),
                  ] # tf.keras.metrics.F1Score() not appropriate as it is calculated batchwise
          )

      print(f'Training for fold {fold_no} ...')

      # Fit the data to the model
      history = model2.fit(
          train.iloc[train_fold], label.iloc[train_fold],
          validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
          batch_size=BATCH_SIZE,
          epochs=5
          )

      # Generate metrics
      scores = model2.evaluate(train.iloc[test_fold], label.iloc[test_fold], verbose=0)
      precision = scores[3]
      recall = scores[4]
      F1_score = 2*precision*recall / (precision + recall)
      print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model2.metrics_names[0]} of {scores[0]}; {model2.metrics_names[1]} of {scores[1]*100}%')

      if F1_score > best_f1:
        best_f1 = F1_score
        if dataset_name == 'Biological Processes':
          tf.keras.models.save_model(
              model2,
              f'{model_root_path}/best_BP_model'
              )
          print(f'Current best model for Biological Processes has {lstm_units} bidirectional LSTM units and an F1 score of {F1_score}')

        elif dataset_name == 'Molecular Function':
          tf.keras.models.save_model(
              model2,
              f'{model_root_path}/best_MF_model'
              )
          print(f'Current best model for Molecular Function has {lstm_units} bidirectional LSTM units and an F1 score of {F1_score}')

        else:
          tf.keras.models.save_model(
              model2,
              f'{model_root_path}/best_CC_model')
          print(f'Current best model for Cellular Component has {lstm_units} bidirectional LSTM units and an F1 score of {F1_score}')

      fold_no += 1

In [ ]:
for dataset in train_data_dict:
  dataset_name = dataset
  data = train_data_dict[dataset]
  model2_training(dataset_name, data)

Training for Biological Processes
Training for fold 1 ...
Epoch 1/5
244/244 [==============================] - 10s 32ms/step - loss: 0.1081 - binary_accuracy: 0.9688 - auc_180: 0.8457 - precision_180: 0.3781 - recall_180: 0.1433
Epoch 2/5
244/244 [==============================] - 8s 31ms/step - loss: 0.0809 - binary_accuracy: 0.9747 - auc_180: 0.9168 - precision_180: 0.6757 - recall_180: 0.2158
Epoch 3/5
244/244 [==============================] - 8s 35ms/step - loss: 0.0769 - binary_accuracy: 0.9752 - auc_180: 0.9280 - precision_180: 0.6822 - recall_180: 0.2467
Epoch 4/5
244/244 [==============================] - 8s 32ms/step - loss: 0.0745 - binary_accuracy: 0.9756 - auc_180: 0.9338 - precision_180: 0.6870 - recall_180: 0.2683
Epoch 5/5
244/244 [==============================] - 7s 31ms/step - loss: 0.0726 - binary_accuracy: 0.9760 - auc_180: 0.9380 - precision_180: 0.6929 - recall_180: 0.2863
Score for fold 1: F1 score of 0.8176819614715455; loss of 0.07005693763494492; binary_accur